## Install neccessary packages

In [2]:
!pip install delta-spark==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 2.8 MB/s eta 0:00:00 0:00:01


## Work with DeltaLake

### Import neccessary packages

In [134]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import round
from delta import *

### Initialize and set up spark to work with DeltaLake

In [52]:
builder = SparkSession.builder.appName("Delta Lake Lab7") \
                              .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                              .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                              .config("spark.sql.catalog.my_catalog.use-nullable-query-schema", "false") \
                              .config("spark.databricks.delta.schema.autoMerge.enabled", "false")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

### Create schema of data and put first data to DeltaLake

In [119]:
# Чомусь обмеження nullable не додавались в DeltaLake при додаванні даних в нього через DataFrame sparka, тому обхідним шляхом створюємо схему даних на пряму
spark.sql("""
CREATE OR REPLACE TABLE delta.`/tmp/delta-traffic` (
  site_id INTEGER NOT NULL,
  visits INTEGER NOT NULL,
  avg_duration FLOAT NOT NULL,
  traffic_source STRING NOT NULL
) USING DELTA;
""")

# Update. Під час оновлення існуючих записів (наприклад, оновити кількість визитів та середній час перегляду) виникла проблема,
# в тому що ми не можемо однозначно ідентифікувати певний запис, оскільки можливо таке, що на певному сайті 100% можуть бути декілька джерел трафіку
# Тому було вирішено зробити поле traffic_source також Not Null, і це поле буде виступати також Первинним ключем в складеному ключі

DataFrame[]

In [120]:
# Створення схеми даних
schema = StructType([
    StructField("site_id", IntegerType(), nullable=False),
    StructField("visits", IntegerType(), nullable=False),
    StructField("avg_duration", FloatType(), nullable=False),
    StructField("traffic_source", StringType(), nullable=False)
])

# Приклад даних для моніторингу трафіку веб-сторінки
data_1 = [
    (1, 50, 3.5, "direct"),
    (1, 100, 3.7, "referral"),
    (1, 75, 4.1, "instagram"),
    (1, 228, 6.4, "ads"),
    (2, 500, 10.1, "direct"),
    (2, 43, 15.3, "referral"),
    (2, 101, 7.3, "instagram"),
    (2, 82, 6.8, "ads")
]

# Створення DataFrame з попередньо ініцілізованих даних
df = spark.createDataFrame(data_1, schema=schema)
df.show()

# Збереження даних у форматі Delta
df.write.format("delta").mode("append").save("/tmp/delta-traffic")

+-------+------+------------+--------------+
|site_id|visits|avg_duration|traffic_source|
+-------+------+------------+--------------+
|      1|    50|         3.5|        direct|
|      1|   100|         3.7|      referral|
|      1|    75|         4.1|     instagram|
|      1|   228|         6.4|           ads|
|      2|   500|        10.1|        direct|
|      2|    43|        15.3|      referral|
|      2|   101|         7.3|     instagram|
|      2|    82|         6.8|           ads|
+-------+------+------------+--------------+



In [121]:
# Читання даних з таблиці Delta
delta_table = spark.read.format("delta").load("/tmp/delta-traffic")
delta_table.show()
delta_table.printSchema()

+-------+------+------------+--------------+
|site_id|visits|avg_duration|traffic_source|
+-------+------+------------+--------------+
|      1|    75|         4.1|     instagram|
|      2|   101|         7.3|     instagram|
|      2|    43|        15.3|      referral|
|      1|   100|         3.7|      referral|
|      2|   500|        10.1|        direct|
|      1|    50|         3.5|        direct|
|      2|    82|         6.8|           ads|
|      1|   228|         6.4|           ads|
+-------+------+------------+--------------+

root
 |-- site_id: integer (nullable = false)
 |-- visits: integer (nullable = false)
 |-- avg_duration: float (nullable = false)
 |-- traffic_source: string (nullable = false)



In [82]:
# Приклад додавання даних з Nullом
data_none_1 = [
    (3, None, 6.8, "ads")
]

data_none_2 = [
    (3, 82, None, "ads")
]

data_none_3 = [
    (None, 82, 6.8, "ads")
]

#df_none_1 = spark.createDataFrame(data_none_1, schema=schema)
#df_none_2 = spark.createDataFrame(data_none_2, schema=schema)
df_none_3 = spark.createDataFrame(data_none_3, schema=schema)

# Навіть створити DataFrame з None значенням не можемо

PySparkValueError: [CANNOT_BE_NONE] Argument `obj` can not be None.

In [122]:
# Приклад оновлення та додавання нових даних
data_2 = [
    (1, 228, 6.4, "ads"),
    (2, 120, 7.1, "instagram"),
    (2, 95, 6.4, "ads"),
    (3, 5, 3.2, "direct")
]

df_2 = spark.createDataFrame(data_2, schema=schema)
df_2.show()

# Створюємо обʼєкт DeltaTable з існуючої таблиці для того, щоб провести оновлення даних за допомогою мерджу
delta_table_2 = DeltaTable.forPath(spark, "/tmp/delta-traffic")

# Проводемо оновлення даних за допомогою мерджу
delta_table_2.alias("web_traffics").merge(
        df_2.alias("new_data"),
            "web_traffics.site_id = new_data.site_id AND web_traffics.traffic_source = new_data.traffic_source"
        ) \
        .whenMatchedUpdate(set={
            "visits": "new_data.visits",
            "avg_duration": "new_data.avg_duration"
        }) \
        .whenNotMatchedInsertAll() \
        .execute()

+-------+------+------------+--------------+
|site_id|visits|avg_duration|traffic_source|
+-------+------+------------+--------------+
|      1|   228|         6.4|           ads|
|      2|   120|         7.1|     instagram|
|      2|    95|         6.4|           ads|
|      3|     5|         3.2|        direct|
+-------+------+------------+--------------+



In [123]:
# Перевіряємо чи дані успішно оновилися
delta_table_2_df = spark.read.format("delta").load("/tmp/delta-traffic")
delta_table_2_df.show()

+-------+------+------------+--------------+
|site_id|visits|avg_duration|traffic_source|
+-------+------+------------+--------------+
|      1|   228|         6.4|           ads|
|      2|    95|         6.4|           ads|
|      2|   120|         7.1|     instagram|
|      3|     5|         3.2|        direct|
|      1|    75|         4.1|     instagram|
|      2|    43|        15.3|      referral|
|      1|   100|         3.7|      referral|
|      2|   500|        10.1|        direct|
|      1|    50|         3.5|        direct|
+-------+------+------------+--------------+



In [124]:
# Перегляд таблиці з історією версій даних
history = delta_table_2.history()
history.show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2025-01-04 23:39:...|  NULL|    NULL|               MERGE|{predicate -> ["(...|NULL|    NULL|     NULL|          1|  Serializable|        false|{numTargetRowsCop...|        NULL|Apache-Spark/3.5....|
|      1|2025-01-04 23:38:...|  NULL|    NULL|               WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          0|  Serializable|         true|{numFiles -

In [137]:
# Повернення до попередньої версії
df_version1 = spark.read.format("delta").option("versionAsOf", 1).load("/tmp/delta-traffic")
df_version2 = spark.read.format("delta").option("versionAsOf", 2).load("/tmp/delta-traffic")

df_version1.show()

# Обрахунок тенденції трафіку
# Виявлення нових записів
new_records = df_version2.join(
    df_version1,
    on=["site_id", "traffic_source"],
    how="leftanti"
)
print("Нові записи у версії 2:")
new_records.show()

# Порівняння існуючих записів
existing_records = df_version2.join(
    df_version1,
    on=["site_id", "traffic_source"],
    how="inner"
).select(
    df_version2["site_id"],
    df_version2["traffic_source"],
    (df_version2["visits"] - df_version1["visits"]).alias("visit_change"),
    round((df_version2["avg_duration"] - df_version1["avg_duration"]), 1).alias("avg_duration_change")
)
print("Тенденція для існуючих записів:")
existing_records.show()


+-------+------+------------+--------------+
|site_id|visits|avg_duration|traffic_source|
+-------+------+------------+--------------+
|      1|    75|         4.1|     instagram|
|      2|   101|         7.3|     instagram|
|      2|    43|        15.3|      referral|
|      1|   100|         3.7|      referral|
|      2|   500|        10.1|        direct|
|      1|    50|         3.5|        direct|
|      2|    82|         6.8|           ads|
|      1|   228|         6.4|           ads|
+-------+------+------------+--------------+

Нові записи у версії 2:
+-------+--------------+------+------------+
|site_id|traffic_source|visits|avg_duration|
+-------+--------------+------+------------+
|      3|        direct|     5|         3.2|
+-------+--------------+------+------------+

Тенденція для існуючих записів:
+-------+--------------+------------+-------------------+
|site_id|traffic_source|visit_change|avg_duration_change|
+-------+--------------+------------+-------------------+
|  

In [138]:
spark.stop()